<a href="https://colab.research.google.com/github/celesica/Mobile-Based-Cacao-Maturity-Detection-Using-YOLOv5/blob/main/Google%2520Colab%2520Notebook/Cacao_Maturity_Detection_(YOLOv5)_Custom_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cacao Maturity Detection**
## Custom Training with YOLOv5 Model

This notebook is based on the [YOLOv5 repository](https://github.com/ultralytics/yolov5) by [Ultralytics](https://www.ultralytics.com/) and [YOLOv5 training blog post](https://blog.roboflow.com/how-to-train-yolov5-on-a-custom-dataset/) by [Roboflow](https://roboflow.com).

### Steps Covered in this Training:

* Install Comet ML for Logging/Visualization
* Install YOLOv5 dependencies
* Download the UF18 Cacao Maturity dataset from Roboflow
* Write the YOLOv5 training configuration
* Run YOLOv5 model training
* Evaluate the model's performance
* Visualize the model's training data
* Run the model's inference on test images
* Export saved YOLOv5 weights to TensorFlow Lite formats


#Step 1: Install Comet ML for Logging/Visualization

Create an account in [Comet ML](https://comet.ml/) if you haven't already, and follow the instructions.

In [ ]:
%pip install comet_ml --quiet

In [ ]:
import comet_ml
comet_ml.init(project_name='cacao-maturity-detection')

#Step 2: Install YOLOv5 Dependencies

In [ ]:
# create the model & config file exporting folder
%mkdir /content/exported_files
INFO_FILE = "/content/exported_files/info.txt"
!touch $INFO_FILE
!echo "Info File:" $INFO_FILE

In [ ]:
# clone YOLOv5 repository
%cd /content
!echo "git clone https://github.com/ultralytics/yolov5" # check the cloning script
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd /content/yolov5
!git log -n 1 # check the latest commit

In [ ]:
%cd /content/yolov5

# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
!pip install -q roboflow
import torch

from IPython.display import Image, clear_output  # to display images

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# Step 3: Download the UF18 Cacao Maturity Dataset

Download the [UF18 Cacao Maturity Dataset](https://universe.roboflow.com/thesiscacaov1/uf18-cacao-maturity) from Roboflow Universe. It uses the "**YOLOv5 PyTorch**" export format. Note that the Ultralytics implementation calls for a YAML file defining where your training and test data is. The Roboflow export also writes this format for us.

In [ ]:
%cd /content/yolov5
#after following the link above, recieve python code with these fields filled in
#from roboflow import Roboflow
#rf = Roboflow(api_key="YOUR API KEY HERE")
#project = rf.workspace().project("YOUR PROJECT")
#dataset = project.version("YOUR VERSION").download("yolov5")
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="API-KEY")
project = rf.workspace("thesiscacaov1").project("uf18-cacao-v3")
dataset = project.version(3).download("yolov5")

In [ ]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

In [ ]:
# define the epoch number, image size & prefered model file for training
EPOCHS = 1000
IMG_SIZE = 640
MODEL = "yolov5s.pt"
MODEL_CONF = "/content/yolov5/models/yolov5s.yaml"

!echo "Epoch:" $EPOCHS >> $INFO_FILE
!echo "Image size:" $IMG_SIZE >> $INFO_FILE
!echo "Base model:" $MODEL >> $INFO_FILE
!echo "Base model config file:" $MODEL_CONF >> $INFO_FILE

# Step 4: Define Model Configuration and Architecture

Write a yaml script that defines the parameters for our model like the number of classes, anchors, and each layer.

In [ ]:
#this is the model configuration
%cat $MODEL_CONF

In [ ]:
# copy the cacao dataset YAML file
%cp $MODEL_CONF /content/yolov5/models/cacaov1_model.yaml


# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:

with open('/content/yolov5/models/cacaov1_model.yaml' , 'r') as f:

    #read file
    file_source = f.read()

    #replace 'nc:' with the num_classes in the file
    new_string = 'nc: '+str(num_classes)+' #'
    replace_string = file_source.replace('nc:', new_string)

with open('/content/yolov5/models/cacaov1_model.yaml', 'w') as f:
    #save output
    f.write(replace_string)

%cat /content/yolov5/models/cacaov1_model.yaml

In [ ]:
# copy the config file to the "exported_files" folder
%cp /content/yolov5/models/cacaov1_model.yaml /content/exported_files
%cp {dataset.location}/data.yaml /content/exported_files

# Step 5: Training the Cacao Maturity YOLOv5 Detector
### Using the YOLOv5s (small) variant

The arguments used for this training are:
- **img:** input image size of 640
- **batch:** Autobatch (-1)
- **epochs:** 1000
- **data:** the path to the yaml file
- **cfg:** the model configuration
- **weights:** the path to weights
- **name:** yolov5_results
- **nosave:** saves every 25th checkpoint
- **cache:** cache images for faster training
- **Comet Logging**

In [ ]:
# download the model file
!wget https://github.com/ultralytics/yolov5/releases/download/v7.0/$MODEL

In [ ]:
# train the "MODEL" on custom data for "EPOCHS" epochs
# time its performance
%%time
%cd /content/yolov5/
!env COMET_MAX_IMAGE_UPLOADS=1000 COMET_LOG_PER_CLASS_METRICS=true python train.py --img $IMG_SIZE --batch-size -1 --epochs $EPOCHS --data {dataset.location}/data.yaml --cfg ./models/cacaov1_model.yaml --weights $MODEL --name yolov5_results --bbox_interval 1 --save-period 25 --cache

# copy the best model to the "exported_files" folder
%cp /content/yolov5/runs/train/yolov5_results/weights/best.pt /content/exported_files

# Step 6: Evaluate Cacao Maturity Classifier Model Performance

Training losses and performance metrics are saved to Tensorboard and also to a logfile defined above with the **--name** flag when we train. In our case, we named this `yolov5_results`. (If given no name, it defaults to `results.txt`.) The results file is plotted as a png after training completes.

Note from Glenn: Partially completed `results.txt` files can be plotted with `from utils.utils import plot_results; plot_results()`.

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason...
%cd /content/yolov5/
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/yolov5_results/results.png', width=1000)  # view results.png

### Visualize Our Training Data with Labels

After training starts, view `train*.jpg` images to see training images, labels and augmentation effects.

Note a mosaic dataloader is used for training (shown below), a new dataloading concept developed by Glenn Jocher and first featured in [YOLOv4](https://arxiv.org/abs/2004.10934).

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TRAINING DATA:")

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/train/yolov5_results/*_batch0_labels.jpg'): #assuming JPG
    display(Image(filename=imageName, width=900))
    break

In [ ]:
# print out an augmented training example
print("GROUND TRUTH AUGMENTED TRAINING DATA:")
Image(filename='/content/yolov5/runs/train/yolov5_results/train_batch0.jpg', width=900)

# Step 7: Run Inference with the Classifier Model
Run inference with a pretrained checkpoint on contents of `test/images` folder downloaded from Roboflow.

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/

In [ ]:
%ls runs/train/yolov5_results/weights

In [ ]:
%cd /content/yolov5/
!python detect.py --weights runs/train/yolov5_results/weights/best.pt --img $IMG_SIZE --conf 0.4 --source {dataset.location}/test/images

In [ ]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('/content/yolov5/runs/detect/exp/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Step 8: Export the Classifier Model to TensorFlow Lite format

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cp /content/exported_files/* /content/gdrive/My\ Drive

In [ ]:
!zip -r /content/exported_files.zip /content/exported_files

In [ ]:
from google.colab import files
files.download('/content/exported_files.zip')

In [ ]:
#export to FP16 TFLite (640 image size)
!python export.py --weights runs/train/yolov5_results/weights/best.pt --include tflite --data /content/exported_files/data.yaml

In [ ]:
#export to INT8 TFLite (640 image size)
!python export.py --weights runs/train/yolov5_results/weights/best.pt --include tflite --int8 --data /content/exported_files/data.yaml

In [ ]:
#export to FP16 TFLite (416 image size)
!python export.py --weights runs/train/yolov5_results/weights/best.pt --img 416 --include tflite --data /content/exported_files/data.yaml

In [ ]:
#export to INT8 TFLite (416 image size)
!python export.py --weights runs/train/yolov5_results/weights/best.pt --include tflite --int8 --img 416 --data /content/exported_files/data.yaml